In [1]:
import os
import cv2
import pandas as pd
import matplotlib.pyplot as plt
from concurrent.futures import ThreadPoolExecutor
import numpy as np
from pathlib import Path

In [2]:
def interpolation(data):
    return(np.interp([i for i in range(1,len(data)*50)],xp=[i*50 for i in range(0,len(data))],fp=data))

In [ ]:
def draw_circle(image,ccol,crow,circle_radius):
    cross_length=10

    circle_img = cv2.circle(image.copy(), (ccol, crow), int(circle_radius), (255, 0, 0), 2)
    cv2.line(circle_img,(ccol - cross_length, crow), (ccol + cross_length, crow), (255, 255, 255), 1)
    cv2.line(circle_img, (ccol, crow - cross_length), (ccol, crow + cross_length), (255, 255, 255), 1)
    
    return circle_img



In [10]:
def process_image_low_movement(filename, output_path, preProcData, result_folder, i,df_list):
    i=i*50
    file_path = os.path.join(output_path, filename)

    image = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)


    movementNearFramesX=preProcData['diff_center_x'][i:i+100].sum()
    movementNearFramesY=preProcData['diff_center_y'][i:i+100].sum()

    if((movementNearFramesX+movementNearFramesY)>10):
        ccol=int(preProcData['center_y(ccol)'][i])
        crow=int(preProcData['center_x(crow)'][i])
    else:
        ccol=int(preProcData['smooth_center_y(ccol)'][i])
        crow=int(preProcData['smooth_center_x(crow)'][i])

    circle_radius = int(preProcData['circle_radius'][i])

    new_data = pd.DataFrame({
        'center_y(ccol)': [ccol],
        'center_x(crow)': [crow],
        'circle_radius': [circle_radius]
    })
    df_list.append(new_data)

    circle_img = draw_circle(image, ccol, crow, circle_radius)
    plt.imshow(circle_img, cmap='gray', vmin=0, vmax=255)
    plt.savefig(os.path.join(result_folder, f"result_{filename}"))
    
    plt.close()
    del circle_img

In [11]:
def process_image_high_movement(filename, output_path, preProcData, result_folder, i,df_list):
    i=i*50
    file_path = os.path.join(output_path, filename)
    
    image = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)

    ccol=int(preProcData['center_y(ccol)'][i])
    crow=int(preProcData['center_x(crow)'][i])
    circle_radius = int(preProcData['circle_radius'][i])

    new_data = pd.DataFrame({
        'center_y(ccol)': [ccol],
        'center_x(crow)': [crow],
        'circle_radius': [circle_radius]
    })
    df_list.append(new_data)

    circle_img = draw_circle(image, ccol, crow, circle_radius)
    plt.imshow(circle_img, cmap='gray', vmin=0, vmax=255)
    plt.savefig(os.path.join(result_folder, f"result_{filename}"))
    plt.close()
    del circle_img

In [14]:

def post_processing_mean(r_in, r_out, ring_index, allowed_change, input_path, input_tiff, output_base_path, interpolationP=True):
    movement_threshold = 3500
    # output_path = Path(output_base_path) / Path(input_tiff).stem
    # print(output_path)
    # output_path.mkdir(parents=True, exist_ok=True)
    test_number = input_tiff.split('.')[1]

    result_folder = f"Results_Validation_Large_Interpolated_Mean/{test_number}/" + Path(input_tiff).stem + f"_Results_bw{r_in}{r_out}_{ring_index}"
    Path(result_folder).mkdir(parents=True, exist_ok=True)

    preProcData=pd.read_excel(f'ExcelsLarge/VideoLarge_object2_{r_in}{r_out}_{ring_index}.xlsx')
    if (interpolationP==True):
        center_x_interpolated=interpolation(preProcData['center_x(crow)'])
        center_y_interpolated=interpolation(preProcData['center_y(ccol)'])
        circle_radius_interpolated=interpolation(preProcData['circle_radius'])
        window=50

        dfList=list(zip(center_x_interpolated,center_y_interpolated,circle_radius_interpolated))
        preProcData=pd.DataFrame(dfList,columns=['center_x(crow)','center_y(ccol)','circle_radius'])
    else:
        window = 3

    # calculating the sum of differences of the center (moving bead or stationary)
    preProcData['diff_center_y'] = preProcData['center_y(ccol)'].diff().abs().fillna(0)
    preProcData['diff_center_x'] = preProcData['center_x(crow)'].diff().abs().fillna(0)
    # preProcData['total_movement'] = preProcData['diff_center_y'] + preProcData['diff_center_x']
    totalMovement=0

    for index, row in preProcData.iterrows():
        totalMovement=totalMovement+row['diff_center_x']+row['diff_center_y']

    preProcData['smooth_center_y(ccol)'] = preProcData['center_y(ccol)'].rolling(window=window).mean()
    preProcData['smooth_center_x(crow)'] = preProcData['center_x(crow)'].rolling(window=window).mean()
    preProcData['smooth_radius'] = preProcData['circle_radius'].rolling(window=window).mean()  

    preProcData.fillna(method='bfill', inplace=True) # for the beginning
    # .rolling(window=3)
    df = pd.DataFrame(columns=['center_y(ccol)', 'center_x(crow)', 'circle_radius'])

    totalMovement = preProcData['diff_center_x'].sum() + preProcData['diff_center_y'].sum()

    df_list=[]

    if totalMovement < movement_threshold:
        with ThreadPoolExecutor(max_workers=10) as executor:
            files = sorted(os.listdir(output_base_path))
            
            for i, filename in enumerate(files):
                executor.submit(process_image_low_movement, filename, output_base_path, preProcData, result_folder, i,df_list)
    else:
        with ThreadPoolExecutor(max_workers=10) as executor:
            files = sorted(os.listdir(output_base_path))
            for i, filename in enumerate(files):
                executor.submit(process_image_high_movement, filename, output_base_path, preProcData, result_folder, i,df_list)

    df = pd.concat(df_list, ignore_index=True)
    # print(df)
    df.to_excel(f"VideoLarge_object2_{r_in}{r_out}_mean.xlsx")


In [15]:
filter_sizes = [(3, 7)] 
tests = range(2, 3)

for filter_size in filter_sizes:
    for test in tests:
        input_path = 'ExcelsLarge'
        input_tiff = f'VideoLarge_object{test}_37_1.xlsx'
        params = {
            'r_in': filter_size[0],
            'r_out': filter_size[1],
            'ring_index': 1,
            'allowed_change': 1,
            'input_path': input_path,
            'input_tiff': input_tiff,
            'output_base_path': f'Results_Validation/VideoLarge/cropped_object_{test}'
        }

        # try:

        post_processing_mean(**params)

        # except Exception as e:
        #     print(f"An error occurred: {e}")
        #     print("Params:", params)
        #     continue

C:\Users\Tautvydas\AppData\Local\Temp\ipykernel_16856\2392921923.py:36: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  preProcData.fillna(method='bfill', inplace=True) # for the beginning


In [17]:
import pandas as pd

# Define paths for the two input Excel files
file_path_1 = 'VideoLarge_object1_37_mean.xlsx'
file_path_2 = 'VideoLarge_object2_37_mean.xlsx'

# Original uncropped image centers for each object
object_1_center = (493, 216)
object_2_center = (332, 730)
half_crop_size = 125  # Since the crop size is 250x250, half_crop_size is 125

# Read both Excel files into DataFrames
df1 = pd.read_excel(file_path_1)
df2 = pd.read_excel(file_path_2)

# Add a frame column if missing, assuming each row corresponds to a frame ID
df1['frame'] = df1.index
df2['frame'] = df2.index

# Rename columns to differentiate between object 1 and object 2 data
df1 = df1.rename(columns={
    'center_x(crow)': '1_x',
    'center_y(ccol)': '1_y',
    'circle_radius': '1_radius'
})
df2 = df2.rename(columns={
    'center_x(crow)': '2_x',
    'center_y(ccol)': '2_y',
    'circle_radius': '2_radius'
})

# Adjust object 1 coordinates to uncropped image coordinates
df1['1_x'] = df1['1_x'] + (object_1_center[0] - half_crop_size)
df1['1_y'] = df1['1_y'] + (object_1_center[1] - half_crop_size)

# Adjust object 2 coordinates to uncropped image coordinates
df2['2_x'] = df2['2_x'] + (object_2_center[0] - half_crop_size)
df2['2_y'] = df2['2_y'] + (object_2_center[1] - half_crop_size)

# Merge the DataFrames on the 'frame' column
merged_df = pd.merge(df1, df2, on='frame')

# Save the merged DataFrame to a new Excel file
merged_df.to_excel('Consolidated_VideoLarge_info_uncropped_coords.xlsx', index=False)

print("Merged data with uncropped coordinates saved to Consolidated_VideoLarge_info_uncropped_coords.xlsx")


Merged data with uncropped coordinates saved to Consolidated_VideoLarge_info_uncropped_coords.xlsx
